## Изградња модела логистичке регресије - Лекција 4

![Инфографика: Логистичка vs. линеарна регресија](../../../../../../2-Regression/4-Logistic/images/linear-vs-logistic.png)

#### **[Квиз пре предавања](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/15/)**

#### Увод

У овој последњој лекцији о регресији, једној од основних *класичних* техника машинског учења, погледаћемо логистичку регресију. Ову технику користите да бисте открили обрасце за предвиђање бинарних категорија. Да ли је овај слаткиш чоколада или не? Да ли је ова болест заразна или не? Да ли ће овај купац изабрати овај производ или не?

У овој лекцији ћете научити:

-   Технике за логистичку регресију

✅ Продубите своје разумевање рада са овом врстом регресије у овом [модулу за учење](https://learn.microsoft.com/training/modules/introduction-classification-models/?WT.mc_id=academic-77952-leestott)

## Предуслов

Радећи са подацима о бундевама, сада смо довољно упознати са њима да схватимо да постоји једна бинарна категорија са којом можемо радити: `Color`.

Хајде да изградимо модел логистичке регресије да бисмо предвидели, на основу неких променљивих, *која је вероватно боја дате бундеве* (наранџаста 🎃 или бела 👻).

> Зашто говоримо о бинарној класификацији у лекцији која се бави регресијом? Само ради језичке погодности, јер је логистичка регресија [заправо метода класификације](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression), иако заснована на линеарним принципима. Сазнајте више о другим начинима класификације података у следећој групи лекција.

За ову лекцију биће нам потребни следећи пакети:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) је [збирка R пакета](https://www.tidyverse.org/packages) дизајнирана да учини науку о подацима бржом, лакшом и забавнијом!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) оквир је [збирка пакета](https://www.tidymodels.org/packages/) за моделирање и машинско учење.

-   `janitor`: [janitor пакет](https://github.com/sfirke/janitor) пружа једноставне алате за испитивање и чишћење "прљавих" података.

-   `ggbeeswarm`: [ggbeeswarm пакет](https://github.com/eclarke/ggbeeswarm) пружа методе за креирање графикона у стилу "роја пчела" користећи ggplot2.

Можете их инсталирати помоћу:

`install.packages(c("tidyverse", "tidymodels", "janitor", "ggbeeswarm"))`

Алтернативно, скрипта испод проверава да ли имате потребне пакете за завршетак овог модула и инсталира их ако недостају.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, ggbeeswarm)


## **Дефинишите питање**

За наше потребе, изразићемо ово као бинарно: 'Бело' или 'Није бело'. У нашем сету података постоји и категорија 'пругасто', али има мало примерака, па је нећемо користити. Ионако нестаје када уклонимо вредности null из сета података.

> 🎃 Забавна чињеница, понекад беле тикве називамо 'духовима' тиквама. Нису баш лаке за резбарење, па нису толико популарне као наранџасте, али изгледају занимљиво! Тако да бисмо могли да преобликујемо наше питање као: 'Дух' или 'Није дух'. 👻

## **О логистичкој регресији**

Логистичка регресија се разликује од линеарне регресије, коју сте раније учили, на неколико важних начина.

#### **Бинарна класификација**

Логистичка регресија не нуди исте могућности као линеарна регресија. Прва нуди предвиђање о `бинарној категорији` ("наранџасто или није наранџасто"), док је друга способна да предвиђа `континуалне вредности`, на пример, на основу порекла тикве и времена жетве, *колико ће њена цена порасти*.

![Инфографик од Дасани Мадипали](../../../../../../2-Regression/4-Logistic/images/pumpkin-classifier.png)

### Друге класификације

Постоје и други типови логистичке регресије, укључујући мултиномијалну и ординалну:

- **Мултиномијална**, која укључује више од једне категорије - "Наранџасто, Бело и Пругасто".

- **Ординална**, која укључује уређене категорије, корисна ако желимо да логички поређамо наше исходе, као тикве које су поређане по ограниченом броју величина (мини, мало, средње, велико, веома велико, екстра велико).

![Мултиномијална vs ординална регресија](../../../../../../2-Regression/4-Logistic/images/multinomial-vs-ordinal.png)

#### **Променљиве НЕ морају да буду корелисане**

Сећате се како је линеарна регресија боље функционисала са више корелисаних променљивих? Логистичка регресија је супротна - променљиве не морају да буду усклађене. То одговара овим подацима који имају релативно слабе корелације.

#### **Потребно је много чистих података**

Логистичка регресија ће дати прецизније резултате ако користите више података; наш мали сет података није оптималан за овај задатак, па то имајте на уму.

✅ Размислите о типовима података који би били погодни за логистичку регресију

## Вежба - уредите податке

Прво, мало очистите податке, уклањајући null вредности и бирајући само неке од колона:

1. Додајте следећи код:


In [ ]:
# Load the core tidyverse packages
library(tidyverse)

# Import the data and clean column names
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv") %>% 
  clean_names()

# Select desired columns
pumpkins_select <- pumpkins %>% 
  select(c(city_name, package, variety, origin, item_size, color)) 

# Drop rows containing missing values and encode color as factor (category)
pumpkins_select <- pumpkins_select %>% 
  drop_na() %>% 
  mutate(color = factor(color))

# View the first few rows
pumpkins_select %>% 
  slice_head(n = 5)


Увек можете бацити поглед на ваш нови датафрејм користећи функцију [*glimpse()*](https://pillar.r-lib.org/reference/glimpse.html) као што је приказано испод:


In [ ]:
pumpkins_select %>% 
  glimpse()


Хајде да потврдимо да ћемо заправо радити на проблему бинарне класификације:


In [ ]:
# Subset distinct observations in outcome column
pumpkins_select %>% 
  distinct(color)


### Визуелизација - категоријални графикон
До сада сте поново учитали податке о бундевама и очистили их тако да задржите скуп података који садржи неколико променљивих, укључујући боју. Хајде да визуелизујемо dataframe у бележници користећи библиотеку ggplot.

Библиотека ggplot нуди неке сјајне начине за визуелизацију ваших података. На пример, можете упоредити расподеле података за сваку сорту и боју у категоријалном графикону.

1. Направите такав графикон користећи функцију geombar, користећи наше податке о бундевама, и одредите мапирање боја за сваку категорију бундеве (наранџаста или бела):


In [ ]:
# Specify colors for each value of the hue variable
palette <- c(ORANGE = "orange", WHITE = "wheat")

# Create the bar plot
ggplot(pumpkins_select, aes(y = variety, fill = color)) +
  geom_bar(position = "dodge") +
  scale_fill_manual(values = palette) +
  labs(y = "Variety", fill = "Color") +
  theme_minimal()

Посматрајући податке, можете видети како се подаци о Боји односе на Врсту.

✅ С обзиром на овај категоријални графикон, која занимљива истраживања можете замислити?


### Предобрада података: кодирање карактеристика

Наш скуп података о бундевама садржи стринг вредности за све своје колоне. Рад са категоријским подацима је интуитиван за људе, али не и за машине. Алгоритми машинског учења добро функционишу са бројевима. Зато је кодирање веома важан корак у фази предобраде података, јер нам омогућава да категоријске податке претворимо у нумеричке, без губитка информација. Добро кодирање води ка изградњи доброг модела.

За кодирање карактеристика постоје два главна типа кодера:

1. Ординални кодер: добро одговара ординалним променљивама, које су категоријске променљиве где њихови подаци следе логички редослед, као што је колона `item_size` у нашем скупу података. Он креира мапирање тако да је свака категорија представљена бројем, који је редослед категорије у колони.

2. Категоријски кодер: добро одговара номиналним променљивама, које су категоријске променљиве где њихови подаци не следе логички редослед, као што су све карактеристике различите од `item_size` у нашем скупу података. То је једно-хот кодирање, што значи да је свака категорија представљена бинарном колоном: кодирана променљива је једнака 1 ако бундева припада тој врсти, а 0 у супротном.

Tidymodels пружа још један користан пакет: [recipes](https://recipes.tidymodels.org/) - пакет за предобраду података. Дефинисаћемо `recipe` који спецификује да све колоне предиктора треба да буду кодиране у сет целих бројева, `prep` да процени потребне количине и статистике за било коју операцију и на крају `bake` да примени израчунавања на нове податке.

> Уобичајено је да се recipes користи као предобрада за моделирање, где дефинише које кораке треба применити на скуп података како би био спреман за моделирање. У том случају је **топло препоручено** да користите `workflow()` уместо ручног процењивања рецепта помоћу prep и bake. Све ово ћемо видети ускоро.
>
> Међутим, за сада користимо recipes + prep + bake да спецификујемо које кораке треба применити на скуп података како би био спреман за анализу података и затим извукли предобрађене податке са примењеним корацима.


In [ ]:
# Preprocess and extract data to allow some data analysis
baked_pumpkins <- recipe(color ~ ., data = pumpkins_select) %>%
  # Define ordering for item_size column
  step_mutate(item_size = ordered(item_size, levels = c('sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo'))) %>%
  # Convert factors to numbers using the order defined above (Ordinal encoding)
  step_integer(item_size, zero_based = F) %>%
  # Encode all other predictors using one hot encoding
  step_dummy(all_nominal(), -all_outcomes(), one_hot = TRUE) %>%
  prep(data = pumpkin_select) %>%
  bake(new_data = NULL)

# Display the first few rows of preprocessed data
baked_pumpkins %>% 
  slice_head(n = 5)


✅ Које су предности коришћења ординалног енкодера за колону величине артикла?

### Анализа односа између променљивих

Сада када смо претходно обрадили наше податке, можемо анализирати односе између карактеристика и ознаке како бисмо стекли идеју о томе колико добро модел може предвидети ознаку на основу карактеристика. Најбољи начин за спровођење овакве анализе је кроз визуализацију података. 
Поново ћемо користити функцију ggplot geom_boxplot_, како бисмо визуализовали односе између величине артикла, сорте и боје у категоријалном графику. За бољу визуализацију података користићемо енкодовану колону величине артикла и неенкодовану колону сорте.


In [ ]:
# Define the color palette
palette <- c(ORANGE = "orange", WHITE = "wheat")

# We need the encoded Item Size column to use it as the x-axis values in the plot
pumpkins_select_plot<-pumpkins_select
pumpkins_select_plot$item_size <- baked_pumpkins$item_size

# Create the grouped box plot
ggplot(pumpkins_select_plot, aes(x = `item_size`, y = color, fill = color)) +
  geom_boxplot() +
  facet_grid(variety ~ ., scales = "free_x") +
  scale_fill_manual(values = palette) +
  labs(x = "Item Size", y = "") +
  theme_minimal() +
  theme(strip.text = element_text(size = 12)) +
  theme(axis.text.x = element_text(size = 10)) +
  theme(axis.title.x = element_text(size = 12)) +
  theme(axis.title.y = element_blank()) +
  theme(legend.position = "bottom") +
  guides(fill = guide_legend(title = "Color")) +
  theme(panel.spacing = unit(0.5, "lines"))+
  theme(strip.text.y = element_text(size = 4, hjust = 0)) 


#### Користите swarm plot

Пошто је Color бинарна категорија (Бела или Не), потребан је 'специјализовани приступ [визуализацији](https://github.com/rstudio/cheatsheets/blob/main/data-visualization.pdf)'.

Пробајте `swarm plot` да бисте приказали расподелу боје у односу на item_size.

Користићемо [ggbeeswarm пакет](https://github.com/eclarke/ggbeeswarm) који пружа методе за креирање beeswarm-стил графикона користећи ggplot2. Beeswarm графикони су начин приказивања тачака које би се иначе преклапале тако да падају једна поред друге.


In [ ]:
# Create beeswarm plots of color and item_size
baked_pumpkins %>% 
  mutate(color = factor(color)) %>% 
  ggplot(mapping = aes(x = color, y = item_size, color = color)) +
  geom_quasirandom() +
  scale_color_brewer(palette = "Dark2", direction = -1) +
  theme(legend.position = "none")


Сада када имамо идеју о односу између бинарних категорија боје и веће групе величина, хајде да истражимо логистичку регресију како бисмо одредили вероватну боју одређене тикве.

## Направите свој модел

Изаберите променљиве које желите да користите у свом класификационом моделу и поделите податке на сетове за обуку и тестирање. [rsample](https://rsample.tidymodels.org/), пакет у оквиру Tidymodels, пружа инфраструктуру за ефикасно раздвајање и ресемплинг података:


In [ ]:
# Split data into 80% for training and 20% for testing
set.seed(2056)
pumpkins_split <- pumpkins_select %>% 
  initial_split(prop = 0.8)

# Extract the data in each split
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)

# Print out the first 5 rows of the training set
pumpkins_train %>% 
  slice_head(n = 5)


🙌 Сада смо спремни да обучимо модел тако што ћемо прилагодити карактеристике за обуку ознаци за обуку (боји).

Почећемо креирањем рецепта који одређује кораке предобраде који треба да се спроведу на нашим подацима како би били спремни за моделирање, тј. кодирање категоријских променљивих у скуп целих бројева. Баш као `baked_pumpkins`, креирамо `pumpkins_recipe`, али не `prep` и `bake`, јер ће бити укључено у радни ток, што ћете видети за само неколико корака.

Постоји прилично велики број начина за спецификацију модела логистичке регресије у Tidymodels. Погледајте `?logistic_reg()`. За сада ћемо специфицирати модел логистичке регресије преко подразумеваног `stats::glm()` механизма.


In [ ]:
# Create a recipe that specifies preprocessing steps for modelling
pumpkins_recipe <- recipe(color ~ ., data = pumpkins_train) %>% 
  step_mutate(item_size = ordered(item_size, levels = c('sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo'))) %>%
  step_integer(item_size, zero_based = F) %>%  
  step_dummy(all_nominal(), -all_outcomes(), one_hot = TRUE)

# Create a logistic model specification
log_reg <- logistic_reg() %>% 
  set_engine("glm") %>% 
  set_mode("classification")


Сада када имамо рецепт и спецификацију модела, потребно је да пронађемо начин да их спојимо у један објекат који ће прво обрадити податке (припрема+печење у позадини), затим обучити модел на обрађеним подацима, а такође омогућити потенцијалне активности пост-обраде.

У Tidymodels-у, овај практични објекат се назива [`workflow`](https://workflows.tidymodels.org/) и згодно чува ваше компоненте за моделирање.


In [ ]:
# Bundle modelling components in a workflow
log_reg_wf <- workflow() %>% 
  add_recipe(pumpkins_recipe) %>% 
  add_model(log_reg)

# Print out the workflow
log_reg_wf


Након што је радни ток *одређен*, модел може бити `обучен` коришћењем функције [`fit()`](https://tidymodels.github.io/parsnip/reference/fit.html). Радни ток ће проценити рецепт и претходно обрадити податке пре обуке, тако да нећемо морати то ручно да радимо користећи prep и bake.


In [ ]:
# Train the model
wf_fit <- log_reg_wf %>% 
  fit(data = pumpkins_train)

# Print the trained workflow
wf_fit


Модел приказује коефицијенте научене током тренинга.

Сада када смо обучили модел користећи податке за тренинг, можемо направити предвиђања на тест подацима користећи [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html). Хајде да започнемо тако што ћемо користити модел за предвиђање ознака за наш тест сет и вероватноћа за сваку ознаку. Када је вероватноћа већа од 0.5, предвиђена класа је `WHITE`, иначе је `ORANGE`.


In [ ]:
# Make predictions for color and corresponding probabilities
results <- pumpkins_test %>% select(color) %>% 
  bind_cols(wf_fit %>% 
              predict(new_data = pumpkins_test)) %>%
  bind_cols(wf_fit %>%
              predict(new_data = pumpkins_test, type = "prob"))

# Compare predictions
results %>% 
  slice_head(n = 10)


Врло лепо! Ово пружа додатне увиде у то како логистичка регресија функционише.

### Боље разумевање кроз матрицу конфузије

Поређење сваке предикције са њеном одговарајућом "истинском" вредношћу није баш ефикасан начин да се утврди колико добро модел предвиђа. Срећом, Tidymodels има још неке трикове у рукаву: [`yardstick`](https://yardstick.tidymodels.org/) - пакет који се користи за мерење ефикасности модела помоћу метрика перформанси.

Једна од метрика перформанси која се повезује са проблемима класификације је [`матрица конфузије`](https://wikipedia.org/wiki/Confusion_matrix). Матрица конфузије описује колико добро модел класификације функционише. Матрица конфузије приказује колико примера у свакој класи је модел исправно класификовао. У нашем случају, показаће вам колико наранџастих бундева је класификовано као наранџасте и колико белих бундева је класификовано као беле; матрица конфузије такође показује колико је њих класификовано у **погрешне** категорије.

Функција [**`conf_mat()`**](https://tidymodels.github.io/yardstick/reference/conf_mat.html) из yardstick-а израчунава ову крос-табелацију посматраних и предвиђених класа.


In [ ]:
# Confusion matrix for prediction results
conf_mat(data = results, truth = color, estimate = .pred_class)


Хајде да интерпретирамо матрицу конфузије. Наш модел треба да класификује бундеве у две бинарне категорије, категорију `бела` и категорију `није бела`.

-   Ако ваш модел предвиди да је бундева бела и она заиста припада категорији „бела“, то називамо `истински позитивно`, што је приказано горњим левим бројем.

-   Ако ваш модел предвиди да бундева није бела, а она заиста припада категорији „бела“, то називамо `лажно негативно`, што је приказано доњим левим бројем.

-   Ако ваш модел предвиди да је бундева бела, а она заиста припада категорији „није бела“, то називамо `лажно позитивно`, што је приказано горњим десним бројем.

-   Ако ваш модел предвиди да бундева није бела, а она заиста припада категорији „није бела“, то називамо `истински негативно`, што је приказано доњим десним бројем.

| Истина |
|:-----:|

|               |        |       |
|---------------|--------|-------|
| **Предвиђено** | БЕЛА  | НАРАНЏАСТА |
| БЕЛА          | TP     | FP    |
| НАРАНЏАСТА    | FN     | TN    |

Као што сте можда претпоставили, пожељно је имати већи број истински позитивних и истински негативних резултата, а мањи број лажно позитивних и лажно негативних резултата, што имплицира да модел боље ради.

Матрица конфузије је корисна јер омогућава израчунавање других метрика које нам могу помоћи да боље проценимо перформансе класификационог модела. Хајде да их размотримо:

🎓 Прецизност: `TP/(TP + FP)` дефинисана као однос предвиђених позитивних резултата који су заиста позитивни. Такође се назива [позитивна предиктивна вредност](https://en.wikipedia.org/wiki/Positive_predictive_value "Positive predictive value").

🎓 Осетљивост: `TP/(TP + FN)` дефинисана као однос позитивних резултата у односу на број узорака који су заиста позитивни. Такође позната као `осетљивост`.

🎓 Специфичност: `TN/(TN + FP)` дефинисана као однос негативних резултата у односу на број узорака који су заиста негативни.

🎓 Тачност: `TP + TN/(TP + TN + FP + FN)` Проценат етикета које су тачно предвиђене за узорак.

🎓 F мера: Тежински просек прецизности и осетљивости, где је најбољи резултат 1, а најгори 0.

Хајде да израчунамо ове метрике!


In [ ]:
# Combine metric functions and calculate them all at once
eval_metrics <- metric_set(ppv, recall, spec, f_meas, accuracy)
eval_metrics(data = results, truth = color, estimate = .pred_class)


## Прикажи РОЦ криву за овај модел

Хајде да направимо још једну визуализацију како бисмо видели такозвану [`РОЦ криву`](https://en.wikipedia.org/wiki/Receiver_operating_characteristic):


In [ ]:
# Make a roc_curve
results %>% 
  roc_curve(color, .pred_ORANGE) %>% 
  autoplot()


ROC криве се често користе за добијање увида у резултате класификатора у смислу његових истинских и лажних позитивних резултата. ROC криве обично приказују `True Positive Rate`/Сензитивност на Y оси и `False Positive Rate`/1-Специфичност на X оси. Због тога су стрмина криве и простор између средишње линије и криве важни: желите криву која брзо иде нагоре и прелази линију. У нашем случају, постоје лажни позитивни резултати на почетку, а затим линија правилно иде нагоре и прелази.

На крају, хајде да користимо `yardstick::roc_auc()` за израчунавање стварне Површине испод криве (Area Under the Curve). Један од начина тумачења AUC је као вероватноћа да модел рангира насумични позитиван пример више него насумични негативан пример.


In [ ]:
# Calculate area under curve
results %>% 
  roc_auc(color, .pred_ORANGE)


Rezultat je oko `0.975`. S obzirom na to da AUC varira od 0 do 1, želite visok rezultat, jer model koji je 100% tačan u svojim predikcijama ima AUC od 1; u ovom slučaju, model je *prilično dobar*.

U budućim lekcijama o klasifikacijama, naučićete kako da poboljšate rezultate svog modela (na primer, kako da se nosite sa neuravnoteženim podacima u ovom slučaju).

## 🚀Изазов

Postoji mnogo toga što treba istražiti u vezi sa logističkom regresijom! Ali najbolji način da naučite jeste da eksperimentišete. Pronađite skup podataka koji se može analizirati ovim tipom modela i napravite model koristeći ga. Шта сте научили? савет: пробајте [Kaggle](https://www.kaggle.com/search?q=logistic+regression+datasets) за занимљиве скупове података.

## Преглед и Самостално Учење

Прочитајте првих неколико страница [овог рада са Стенфорда](https://web.stanford.edu/~jurafsky/slp3/5.pdf) о неким практичним применама логистичке регресије. Размислите о задацима који су боље прилагођени једном или другом типу регресије које смо до сада проучавали. Шта би најбоље функционисало?



---

**Одрицање од одговорности**:  
Овај документ је преведен коришћењем услуге за превођење помоћу вештачке интелигенције [Co-op Translator](https://github.com/Azure/co-op-translator). Иако настојимо да обезбедимо тачност, молимо вас да имате у виду да аутоматизовани преводи могу садржати грешке или нетачности. Оригинални документ на изворном језику треба сматрати ауторитативним извором. За критичне информације препоручује се професионални превод од стране људи. Не сносимо одговорност за било каква погрешна тумачења или неспоразуме који могу произаћи из коришћења овог превода.
